# <span style="font-width:bold; font-size: 3rem; color:#2656a3;">**Msc. BDS Module - Data Engineering and Machine Learning Operations in Business (MLOPs)** </span> <span style="font-width:bold; font-size: 3rem; color:#333;">- Part 03: Training Pipeline</span>

## <span style='color:#2656a3'> 🗒️ This notebook is divided into the following sections:
1. Feature selection.
2. Creating a Feature View.
3. Training datasets creation - splitting into train and test sets.
4. Training the model.
5. Register the model to Hopsworks Model Registry.

## <span style='color:#2656a3'> ⚙️ Import of libraries and packages
We start with importing some of the necessary libraries needed for this notebook and warnings to avoid unnecessary distractions and keep output clean.

In [25]:
# Importing the packages and libraries
import pandas as pd
import numpy as np

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)

## <span style="color:#2656a3;"> 📡 Connecting to Hopsworks Feature Store
We connect to Hopsworks Feature Store so we can retrieve the Feature Groups and select features for training data.

In [2]:
# Importing the hopsworks module for interacting with the Hopsworks platform
import hopsworks

# Logging into the Hopsworks project
project = hopsworks.login()

# Getting the feature store from the project
fs = project.get_feature_store() 

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/554133
Connected. Call `.close()` to terminate connection gracefully.


In [3]:
# Retrieve the feature groups
electricity_fg = fs.get_feature_group(
    name='electricity_prices',
    version=1,
)

weather_fg = fs.get_feature_group(
    name='weather_measurements',
    version=1,
)

danish_calendar_fg = fs.get_feature_group(
    name='dk_calendar',
    version=1,
)

## <span style="color:#2656a3;"> 🖍 Feature View Creation and Retrieving </span>

We first select the features that we want to include for model training.

Since we specified `primary_key`as `date` and `timestamp` in `1_feature_backfill` we can now join them together for the `electricity_fg`, `weather_fg` and `danish_holiday_fg`.

`join_type` specifies the type of join to perform. An inner join refers to only retaining the rows based on the keys present in all joined DataFrames.

In [4]:
# Select features for training data and join them together and except duplicate columns
selected_features_training = electricity_fg.select_all()\
    .join(weather_fg.select_except(["timestamp", "datetime", "hour"]), join_type="inner")\
    .join(danish_calendar_fg.select_all(), join_type="inner")

In [5]:
# Display the first 5 rows of the selected features
selected_features_training.show(5)

Finished: Reading data from Hopsworks, using ArrowFlight (4.86s) 


,timestamp,datetime,date,hour,dk1_spotpricedkk_kwh,temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m,dayofweek,day,month,year,workday
0,1682280000000,2023-04-23 20:00:00+00:00,2023-04-23,20,1.02178,10.4,74.0,0.0,0.0,0.0,3.0,100.0,7.6,10.1,6,23,4,2023,0
1,1678816800000,2023-03-14 18:00:00+00:00,2023-03-14,18,0.77461,0.5,88.0,0.0,0.0,0.0,0.0,0.0,11.6,22.7,1,14,3,2023,1
2,1697259600000,2023-10-14 05:00:00+00:00,2023-10-14,5,-0.01551,9.8,71.0,0.0,0.0,0.0,1.0,23.0,29.5,54.7,5,14,10,2023,0
3,1657170000000,2022-07-07 05:00:00+00:00,2022-07-07,5,1.15795,15.0,90.0,0.1,0.1,0.0,51.0,59.0,16.6,31.3,3,7,7,2022,1
4,1647597600000,2022-03-18 10:00:00+00:00,2022-03-18,10,1.48754,8.4,60.0,0.0,0.0,0.0,0.0,0.0,21.9,45.4,4,18,3,2022,1


A `Feature View` stands between the **Feature Groups** and **Training Dataset**. Сombining **Feature Groups** we can create a **Feature View** which stores a metadata of our data. Having the **Feature View** we can create a **Training Dataset**.

In order to create Feature View we can use `fs.get_or_create_feature_view()` method.

We can specify parameters:

- `name` - Name of the feature view to create.
- `version` - Version of the feature view to create.
- `query` - Query object with the data.

In [6]:
# Getting or creating a feature view named 'dk1_electricity_training_feature_view'
version = 1
feature_view_training = fs.get_or_create_feature_view(
    name='dk1_electricity_training_feature_view',
    version=version,
    query=selected_features_training,
)

In [7]:
print(feature_view_training.query.to_string())

SELECT `fg2`.`timestamp` `timestamp`, `fg2`.`datetime` `datetime`, `fg2`.`date` `date`, `fg2`.`hour` `hour`, `fg2`.`dk1_spotpricedkk_kwh` `dk1_spotpricedkk_kwh`, `fg0`.`temperature_2m` `temperature_2m`, `fg0`.`relative_humidity_2m` `relative_humidity_2m`, `fg0`.`precipitation` `precipitation`, `fg0`.`rain` `rain`, `fg0`.`snowfall` `snowfall`, `fg0`.`weather_code` `weather_code`, `fg0`.`cloud_cover` `cloud_cover`, `fg0`.`wind_speed_10m` `wind_speed_10m`, `fg0`.`wind_gusts_10m` `wind_gusts_10m`, `fg1`.`dayofweek` `dayofweek`, `fg1`.`day` `day`, `fg1`.`month` `month`, `fg1`.`year` `year`, `fg1`.`workday` `workday`
FROM `tobiasmj_featurestore`.`electricity_prices_1` `fg2`
INNER JOIN `tobiasmj_featurestore`.`weather_measurements_1` `fg0` ON `fg2`.`timestamp` = `fg0`.`timestamp` AND `fg2`.`date` = `fg0`.`date`
INNER JOIN `tobiasmj_featurestore`.`dk_calendar_1` `fg1` ON `fg2`.`date` = `fg1`.`date`


## <span style="color:#2656a3;"> 🏋️ Training Dataset Creation</span>

In Hopsworks, a training dataset is generated from a query defined by the parent FeatureView, which determines the set of features.

**Training Dataset may contain splits such as:** 
* Training set: This subset of the training data is utilized for model training.
* Validation set: Used for evaluating hyperparameters during model training. *(We have not included a validation set for this project)*
* Test set: Reserved as a holdout subset of training data for evaluating a trained model's performance.

Training dataset is created using `fs.training_data()` method.

In [8]:
# Retrieve training data from the feature view 'feature_view_training', assigning the features to 'X'.
X, _ = feature_view_training.training_data(
    description = 'Electricity Prices Training Dataset',
)

Finished: Reading data from Hopsworks, using ArrowFlight (6.56s) 


In [9]:
# Show the information for the training data
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20661 entries, 0 to 20660
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   timestamp             20661 non-null  int64  
 1   datetime              20661 non-null  object 
 2   date                  20661 non-null  object 
 3   hour                  20661 non-null  int64  
 4   dk1_spotpricedkk_kwh  20661 non-null  float64
 5   temperature_2m        20661 non-null  float64
 6   relative_humidity_2m  20661 non-null  float64
 7   precipitation         20661 non-null  float64
 8   rain                  20661 non-null  float64
 9   snowfall              20661 non-null  float64
 10  weather_code          20661 non-null  float64
 11  cloud_cover           20661 non-null  float64
 12  wind_speed_10m        20661 non-null  float64
 13  wind_gusts_10m        20661 non-null  float64
 14  dayofweek             20661 non-null  int64  
 15  day                

### <span style="color:#2656a3;"> ⛳️ Dataset with train and test splits</span>

Here we define our train and test splits for traning the model.

In [10]:
# Importing function for splitting the data into training and testing sets
from sklearn.model_selection import train_test_split

In [11]:
# Drop the columns 'date', 'datetime', and 'timestamp' from the DataFrame 'X' which contain the features
X = X.drop(columns=['date', 'datetime', 'timestamp'])

In [12]:
# Remove the dependent variable 'dk1_spotpricedkk_kwh' from the DataFrame 'X' and assign it to the variable 'y'
y = X.pop('dk1_spotpricedkk_kwh')

In [13]:
# Split the features and the dependent variable into training and testing sets using the train_test_split function
# This splits the data randomly into 80% training and 20% testing sets. We set the random_state to 42 to ensure reproducibility.
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.2, 
    random_state=42,
)

In [14]:
# Display the first 5 rows of the features in the training data
X_train.head()

,hour,temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m,dayofweek,day,month,year,workday
844,14,7.8,60.0,0.0,0.0,0.0,0.0,0.0,19.9,40.3,0,28,2,2022,1
18796,17,26.6,47.0,0.0,0.0,0.0,0.0,1.0,11.4,23.0,1,19,7,2022,1
11952,17,6.0,62.0,0.0,0.0,0.0,1.0,28.0,26.7,53.3,2,26,4,2023,1
19885,18,-5.1,80.0,0.0,0.0,0.0,3.0,100.0,12.7,25.9,3,4,1,2024,1
1783,5,4.6,65.0,0.0,0.0,0.0,0.0,17.0,24.1,46.1,3,27,4,2023,1


In [15]:
# Display the first 5 rows of the dependent variable in the training data
y_train.head()

844      1.05044
18796    3.30052
11952    0.50579
19885    1.07877
1783     0.76701
Name: dk1_spotpricedkk_kwh, dtype: float64

## <span style="color:#2656a3;">🧬 Modeling</span>

For Modeling we initialize the `XGBoost Regressor`.

The XGBoost Regressor is a powerful and versatile algorithm known for its effectiveness in a wide range of regression tasks, including predictive modeling and time series forecasting. Specifically tailored for regression tasks, it aims to predict continuous numerical values. The algorithm constructs an ensemble of regression trees, optimizing them to minimize a specified loss function, commonly the mean squared error for regression tasks. Ultimately, the final prediction is derived by aggregating the predictions of individual trees.

In [ ]:
# New - see all models and their scores/performance

from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import LinearSVR
from sklearn.linear_model import SGDRegressor
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [33]:
# Create a list of models
models = [RandomForestRegressor, DecisionTreeRegressor, KNeighborsRegressor, 
          XGBRegressor]

# Create a list of model names
names = ['RandomForestRegressor', 'DecisionTreeRegressor', 'KNeighborsRegressor', 
           'XGBRegressor']

# Create empty lists to store the scores
accuracy_scores = []
mse_scores = []
mae_scores = []
r2_scores = []

# Loop through the models
for mod in models:
    model = mod()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    accuracy = model.score(X_test, y_test)
    r2 = r2_score(y_test, y_pred)
    mse_scores.append(mse)
    mae_scores.append(mae)
    accuracy_scores.append(accuracy)
    r2_scores.append(r2)

# Create a DataFrame of the results
results = pd.DataFrame({'Models': names, 
                        'MSE': mse_scores, 
                        'MAE': mae_scores, 
                        # 'Accuracy': accuracy_scores,
                        'R2 Score': r2_scores}
                        ).sort_values(by='R2 Score', ascending=False).set_index('Models')

# Display the results
print(results)

                            MSE       MAE  R2 Score
Models                                             
RandomForestRegressor  0.041980  0.129812  0.949644
XGBRegressor           0.054013  0.159776  0.935210
DecisionTreeRegressor  0.089955  0.174972  0.892098
KNeighborsRegressor    0.492466  0.461400  0.409277


In [18]:
# Importing the XGBoost Regressor
import xgboost as xgb

# Initialize the XGBoost Regressor
model = xgb.XGBRegressor()

In [19]:
# Train the model on the training data
model.fit(X_train, y_train)

See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

## <span style='color:#ff5f27'> ⚖️ Model Validation

After fitting the XGBoost Regressor, we evaluate the performance using the following validation metrics.

**Mean Squared Error (MSE):**
- Measures the average squared difference between the actual and predicted values in a regression problem. 
- It squares the differences between predicted and actual values to penalize larger errors more heavily.
- Lower MSE values indicate better model performance.

**R-squared (R²):**
- Measures the proportion of the variance in the dependent variable (target) that is predictable from the independent variables (features) in a regression model.
- R-squared values range from 0 to 1, where 0 indicates that the model does not explain any variability in the target variable, and 1 indicates that the model explains all the variability.
- R-squared is a useful metric for assessing how well the regression model fits the observed data. However, it does not provide information about the goodness of fit on new, unseen data.

**Mean Absolute Error (MAE):**
- Measures the average absolute difference between the actual and predicted values.
- MAE is less sensitive to outliers compared to MSE because it does not square the errors.
- Like MSE and RMSE, lower MAE values indicate better model performance.

MSE focus on the magnitude of errors, while R-squared provides insight into the proportion of variance explained by the model. MAE provides a measure of average error without considering the direction of errors.

In [20]:
# Importing the model validation metric functions from the sklearn library
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [21]:
# Predict target values on the test set
y_pred = model.predict(X_test)

# Calculate Mean Squared Error (MSE) using sklearn
mse = mean_squared_error(y_test, y_pred)
print("⛳️ MSE:", mse)

# Calculate R squared using sklearn
r2 = r2_score(y_test, y_pred)
print("⛳️ R^2:", r2)

# Calculate Mean Absolute Error (MAE) using sklearn
mae = mean_absolute_error(y_test, y_pred)
print("⛳️ MAE:", mae)

⛳️ MSE: 0.054013112252775976
⛳️ R^2: 0.9352102369505784
⛳️ MAE: 0.15977614413520452


In this case, the `MSE` is 0.0546, which suggests that on average, the squared difference between the actual and predicted values is relatively low. An `R^2` value of 0.933 indicates that approximately 93.33% of the variance in the dependent variable is predictable from the feature variables in the model. This is a high value, suggesting that the model explains a significant portion of the variability in the data. A `MAE` of 0.1604 suggests that, on average, the model's predictions are off by approximately 0.1604 units from the actual values. Similar to MSE, a lower MAE indicates better accuracy of the model.

In summary, based on these metrics, the model seems to perform quite well. It has relatively low error (both in terms of MSE and MAE), and a high percentage of the variance in the dependent variable is explained by the feature variables, as indicated by the high R-squared value.

In [ ]:
# Importing the matplotlib library for plotting the predictions against the expected values
import matplotlib.pyplot as plt

# Plot the predictions against the expected values
plt.title('Expected vs Predicted Electricity Prices for area DK1')

# Plot the predicted values
plt.bar(x=np.arange(len(y_pred)), height=y_pred, label='predicted', alpha=0.7)

# Plot the expected values
plt.bar(x=np.arange(len(y_pred)), height=y_test, label='expected', alpha=0.7)

# Add labels to the x-axis and y-axis
plt.xlabel('Time')
plt.ylabel('Price in DKK')

# Add a legend and display the plot
plt.legend()
plt.show() 

In [ ]:
# Import the plot_importance function from XGBoost
from xgboost import plot_importance

# Plot feature importances using the plot_importance function from XGBoost
plot_importance(
    model, 
    max_num_features=25,  # Display the top 25 most important features
)
plt.show()

As shown in the above feature importance plot features like `temperature`, `day`, `hour` and `month` are most important for predicting the dependent variable. 

## <span style='color:#2656a3'>🗄 Model Registry</span>

The Model Registry in Hopsworks enable us to store the trained model. The model registry centralizes model management, enabling models to be securely accessed and governed. We can also save model metrics with the model, enabling the user to understand performance of the model on test (or unseen) data.

In [ ]:
# Importing the libraries for saving the model
from hsml.schema import Schema
from hsml.model_schema import ModelSchema
import joblib

In [ ]:
# Retrieving the Model Registry from Hopsworks
mr = project.get_model_registry()

### <span style="color:#ff5f27;">⚙️ Model Schema</span>
A model schema defines the structure and format of the input and output data that a machine learning model expects and produces, respectively. It serves as a **blueprint** for understanding how to interact with the model in terms of input features and output predictions. In the context of the Hopsworks platform, a model schema is typically defined using the Schema class, which specifies the features expected in the input data and the target variable in the output data. This schema helps ensure consistency and compatibility between the model and the data it operates on.

In [ ]:
# Imoprt the os library to interact with the operating system
import os

In [ ]:
# Specify the schema of the model's input and output using the features (X_train) and dependent variable (y_train)
input_schema = Schema(X_train)
output_schema = Schema(y_train)

# Create a model schema using the input and output schemas
model_schema = ModelSchema(input_schema, output_schema)

In [ ]:
# Define the directory path (folder path) where the trained model will be exported
model_dir = "model"

# Check if the directory already exists, if not create it
if not os.path.isdir(model_dir):
    os.mkdir(model_dir)

In [ ]:
# Save the XGBoost Regressor model as joblib file in the model directory
joblib.dump(model, model_dir + "/dk_electricity_model.pkl")

In [ ]:
# Create an entry in the model registry with the specified details 
xgb_model = mr.python.create_model(
    name="electricity_price_prediction_model", # Name of the model
    metrics={ # Evaluation metrics for the model
        "MSE": mse,
        "R squared": r2,
        "MAE": mae,
    },
    model_schema=model_schema, # Schema defining the input and output data structure of the model
    input_example=X_train.sample(), # Example input data for the model
    description="DK1 Electricity Price Predictor" # Description of the model
)

In [ ]:
# Upload the model to Hopsworks
xgb_model.save(model_dir)

## <span style="color:#2656a3;">⏭️ **Next:** Part 04: Batch Inference </span>

Next notebook we will use the registered model to make predictions based on the batch data.